# **Recurrent Neural Network: Time Series Forecasting**
**Artificial  Neural  Networks  and  Deep  Learning  -  a.y.  2021/2022**

*     <u>Fabio Tresoldi</u>
> M.Sc. Computer Science and Engineering
>
> Politecnico di Milano - Milan, Italy
>
> E-mail: fabio1.tresoldi@mail.polimi.it
>
> Student ID : 10607540
>
> Codalab Nickname: "fabioow"
>
> Codalab Group: "artificial_comrades"

*     <u>Mirko  Usuelli</u>
> M.Sc. Computer Science and Engineering
>
> Politecnico di Milano - Milan, Italy
>
>E-mail: mirko.usuelli@mail.polimi.it
>
>Student ID : 10570238
>
>Codalab Nickname: "mirko"
>
>Codalab Group: "artificial_comrades"


## Environment Settings

### Libraries

In [54]:
import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt
plt.rc('font', size=16)

import warnings
warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

### Random Seed

In [55]:
# Random seed for reproducibility
SEED = 42

np.random.seed(SEED)
tf.random.set_seed(SEED)
tf.compat.v1.set_random_seed(SEED)

## Additive Decomposition AutoEncoder

### Data Loading

In [56]:
dataset = pd.read_csv('Training.csv')
print(dataset.shape)
dataset.head()

In [57]:
dataset.info()

### Data Visualization

In [58]:
FEATURES = ['Sponginess',
            'Wonder level',
            'Crunchiness',
            'Loudness on impact',
            'Meme creativity',
            'Soap slipperiness',
            'Hype root']

_ = dataset[FEATURES].plot(subplots=True, figsize=(25,18))

### Data Preprocessing

#### Sequentila Train-Test Split

In [59]:
DATA_SIZE = len(dataset)
TEST_SIZE = int(DATA_SIZE * 0.1)
NUM_FEATURES = dataset.shape[1]

train_df = dataset.iloc[:-TEST_SIZE]
test_df = dataset.iloc[-TEST_SIZE:]

print('---------------------------')
print('DATA SIZE :', DATA_SIZE)
print('FEATURES :', NUM_FEATURES)
print('---------------------------')
print('TEST SIZE :', TEST_SIZE)
print('TEST SHAPE :', test_df.shape)
print('---------------------------')
print('TRAIN SIZE :', DATA_SIZE - TEST_SIZE)
print('TRAIN SHAPE :', train_df.shape)
print('---------------------------')

#### Normalization

In [60]:
# Normalize both features and labels
X_min = train_df.min()
X_max = train_df.max()

train_df = (train_df - X_min) / (X_max - X_min)
test_df = (test_df - X_min) / (X_max - X_min)

In [61]:
for feat in FEATURES:
    plt.figure(figsize=(25,3))
    plt.plot(train_df[feat], label='Train (' + feat + ')')
    plt.plot(test_df[feat], label='Test (' + feat + ')')
    plt.title('Train-Test Split (' + feat + ')')
    plt.legend()
    plt.show()

#### Train, Test and Future Sequences Generation

In [62]:
WINDOW = 200
STRIDE = 10
TELESCOPE = 864

future = (dataset[-WINDOW:] - X_min) / (X_max - X_min)
future = np.expand_dims(future, axis=0)

print('---------------------------')
print('WINDOW SIZE :', WINDOW)
print('STRIDE :', STRIDE)
print('FUTURE SHAPE :', future.shape)
print('TELESCOPE :', TELESCOPE)
print('---------------------------')

In [63]:
def build_sequences(df, target_labels=FEATURES, window=WINDOW, stride=STRIDE, telescope=TELESCOPE):
    # Sanity check to avoid runtime errors
    assert window % stride == 0
    dataset = []
    labels = []
    temp_df = df.copy().values
    temp_label = df[target_labels].copy().values
    padding_len = len(df)%window

    if(padding_len != 0):
        # Compute padding length
        padding_len = window - len(df)%window
        padding = np.zeros((padding_len,temp_df.shape[1]), dtype='float64')
        temp_df = np.concatenate((padding,df))
        padding = np.zeros((padding_len,temp_label.shape[1]), dtype='float64')
        temp_label = np.concatenate((padding,temp_label))
        assert len(temp_df) % window == 0

    for idx in np.arange(0,len(temp_df)-window-telescope,stride):
        dataset.append(temp_df[idx:idx+window])
        labels.append(temp_label[idx+window:idx+window+telescope])

    dataset = np.array(dataset)
    labels = np.array(labels)
    return dataset, labels

In [64]:
X_train, y_train = build_sequences(train_df)
X_test, y_test = build_sequences(test_df)

print("TRAINIG:")
print("- Sequence shapes:")
print(X_train.shape, y_train.shape)

print("\n")
print("TESTING:")
print("- Sequence shapes:")
print(X_test.shape, y_test.shape)

In [65]:
def inspect_multivariate(X, y, columns=FEATURES, telescope=TELESCOPE, idx=None):
    if(idx == None):
        rng = np.random.RandomState(SEED)
        idx = rng.randint(0,len(X))

    figs, axs = plt.subplots(len(columns), 1, sharex=True, figsize=(17,17))
    for i, col in enumerate(columns):
        axs[i].plot(np.arange(len(X[0,:,i])), X[idx,:,i])
        axs[i].scatter(np.arange(len(X[0,:,i]), len(X_train[0,:,i])+telescope), y[idx,:,i], color='orange')
        axs[i].set_title(col)
        axs[i].set_ylim(0,1)
    plt.show()

In [66]:
inspect_multivariate(X_train, y_train)

### AutoEncoder Skeleton

In [67]:
MODEL_INPUT_SHAPE = X_train.shape[1:]
INPUT_SHAPE = X_train.shape[1:]
OUTPUT_SHAPE = y_train.shape[1:]

VALIDATION_SPLIT = 0.1
BATCH_SIZE = 64
EPOCHS = 200

OUTPUT_SHAPE

#### Encoder

In [68]:
ENCODER_DIM = 128

In [69]:
def build_encoder(name, input_shape=INPUT_SHAPE):
    encoder_input = tfkl.Input(shape=input_shape, name='Encoder_Input')
    x = tfkl.Conv1D(int(ENCODER_DIM / 2), 3, padding='same', activation='relu', name='Conv_1')(encoder_input)
    x = tfkl.MaxPool1D(name='MaxPool_1')(x)
    x = tfkl.Conv1D(ENCODER_DIM, 3, padding='same', activation='relu', name='Conv_2')(x)
    x = tfkl.MaxPool1D(name='MaxPool_2')(x)
    x = tfkl.Bidirectional(tfkl.LSTM(ENCODER_DIM, kernel_initializer='he_uniform', return_sequences=True, bias_initializer="ones", stateful=False, name='Bi-LSTM_1'))(x)
    encoder_output = tfkl.Bidirectional(tfkl.LSTM(int(ENCODER_DIM / 2), kernel_initializer='he_uniform', return_sequences=False, bias_initializer="ones", stateful=False, name='Encoder_Output'))(x)

    encoder = tfk.Model(encoder_input, encoder_output, name='Encoder_'+str(name))
    return encoder

In [70]:
# Encoder Summary Example
encoder = build_encoder(name="sample")
encoder.summary()
tfk.utils.plot_model(encoder, expand_nested=True)

#### Decoder

In [71]:
def build_decoder():
    decoder_input= tfkl.Input(shape=(WINDOW, ENCODER_DIM), name='Decoder_Input')
    x = tfkl.Bidirectional(tfkl.LSTM(64, kernel_initializer='he_uniform', return_sequences=True, bias_initializer="ones", stateful=False, name='Bi-LSTM_1'))(decoder_input)
    x = tfkl.Bidirectional(tfkl.LSTM(128, kernel_initializer='he_uniform', return_sequences=True, bias_initializer="ones", stateful=False, name='Bi-LSTM_2'))(x)
    decoder_output = tfkl.TimeDistributed(tfkl.Dense(7), name='Decoder_Output')(x)

    decoder = tfk.Model(decoder_input, decoder_output, name='Decoder')
    return decoder

In [72]:
# Decoder Summary Example
decoder = build_decoder()
decoder.summary()
tfk.utils.plot_model(decoder, expand_nested=True)

#### AutoEncoder

In [73]:
def build_autoencoder(encoder, decoder, input_shape=INPUT_SHAPE):
    auto_input = tfkl.Input(shape=input_shape)
    encoded = encoder(auto_input)
    bridge = tfkl.RepeatVector(200, name='encoder_decoder_bridge')(encoded)
    decoded = decoder(bridge)

    autoencoder = tfk.Model(auto_input, decoded, name='Autoencoder')
    return autoencoder

In [74]:
# AutoEncoder Summary Example
autoencoder = build_autoencoder(encoder, decoder)
autoencoder.summary()
tfk.utils.plot_model(autoencoder, expand_nested=True)

##### Training

In [75]:
encoder = build_encoder(name="Encoder")
autoencoder = build_autoencoder(encoder, build_decoder())
autoencoder.compile(loss=tfk.losses.MeanSquaredError(), optimizer=tfk.optimizers.Adam(), metrics=['mae'])

In [76]:
# Train the autoencoder
autoencoder.fit(
    x = X_train,
    y = X_train, # AutoEncoder target is its own input
    batch_size = BATCH_SIZE,
    epochs = EPOCHS,
    validation_split = VALIDATION_SPLIT,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-5)
    ]
)

#### Encoder Trainable Layers

Freeze all the layers except the last one, which is going to be fine tuned

In [92]:
print("BEFORE:")
for l in encoder.layers:
    print(l.name, l.trainable)
    
for l in encoder.layers:
    l.trainable = False

print("\n")
print("AFTER:")
encoder.layers[-1].trainable = True
for l in encoder.layers:
    print(l.name, l.trainable)

### Forecaster Skeleton

#### Forecaster

In [78]:
def build_forecaster(name, output_shape=OUTPUT_SHAPE):
    forecaster_input = tfkl.Input(shape=ENCODER_DIM, name='Forecaster_Input')
    x = tfkl.Dropout(.3, seed=SEED, name='DropOut')(forecaster_input)
    x = tfkl.Dense(output_shape[-1]*output_shape[-2], activation='relu', name='Dense')(x)
    forecaster_output = tfkl.Reshape((output_shape[-2],output_shape[-1]), name='Forecaster_Output')(x)

    # Connect input and output through the Model class
    forecaster = tfk.Model(forecaster_input, forecaster_output, name='Forecaster_'+str(name))
    
    return forecaster

In [79]:
forecaster = build_forecaster(name='Forecaster')

In [80]:
# Forecaster Summary Example
forecaster.summary()
tfk.utils.plot_model(forecaster, expand_nested=True)

### AutoEncoder + Forecaster

In [81]:
model_input = tfkl.Input(shape=MODEL_INPUT_SHAPE, name='Model_Input')

input_layer = tfkl.Lambda(lambda x: x[:, :, :7])(model_input)
reshape_layer = tfkl.Reshape((input_layer.shape[1], 7))(input_layer)
encoded_layer = encoder(reshape_layer)
forecasted_layer = forecaster(encoded_layer)

model_output = tfkl.Conv1D(OUTPUT_SHAPE[-1], 1, padding='same', name='Model_Output')(forecasted_layer)

# Connect input and output through the Model class
model = tfk.Model(model_input, model_output, name='Encoder_Forecaster')

In [82]:
# Compile the model
model.compile(loss=tfk.losses.MeanSquaredError(), optimizer=tfk.optimizers.Adam(), metrics=['mae'])

# Summary
model.summary()

# Draw
tfk.utils.plot_model(model, expand_nested=True)

### Training (Fine Tuning)

In [83]:
# Train the model
history = model.fit(
    x = X_train,
    y = y_train,
    batch_size = BATCH_SIZE,
    epochs = EPOCHS,
    validation_split = VALIDATION_SPLIT,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-5)
    ]
).history

In [84]:
best_epoch = np.argmin(history['val_loss'])
plt.figure(figsize=(17,4))
plt.plot(history['loss'], label='Training loss', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation loss', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Squared Error (Loss)')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(17,4))
plt.plot(history['mae'], label='Training accuracy', alpha=.8, color='#ff7f0e')
plt.plot(history['val_mae'], label='Validation accuracy', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Absolute Error')
plt.legend()
plt.grid(alpha=.3)
plt.show()

### Direct Predictions

In [85]:
# Predict the test set 
predictions = model.predict(X_test)
print(predictions.shape)

mean_squared_error = tfk.metrics.mse(y_test.flatten(),predictions.flatten())
mean_absolute_error = tfk.metrics.mae(y_test.flatten(),predictions.flatten())
mean_squared_error, mean_absolute_error

In [86]:
def inspect_multivariate_prediction(X, y, pred, columns=FEATURES, telescope=TELESCOPE, idx=None):
    if(idx == None):
        rng = np.random.RandomState(SEED)
        idx = rng.randint(0,len(X))

    figs, axs = plt.subplots(len(columns), 1, sharex=True, figsize=(17,17))
    for i, col in enumerate(columns):
        axs[i].plot(np.arange(len(X[0,:,i])), X[idx,:,i])
        axs[i].plot(np.arange(len(X[0,:,i]), len(X_train[0,:,i])+telescope), y[idx,:,i], color='orange')
        axs[i].plot(np.arange(len(X[0,:,i]), len(X_train[0,:,i])+telescope), pred[idx,:,i], color='green')
        axs[i].set_title(col)
        axs[i].set_ylim(0,1)
    plt.show()

In [87]:
inspect_multivariate_prediction(X_test, y_test, predictions)

### Future Predictions

In [88]:
maes = []
for i in range(predictions.shape[1]):
    ft_maes = []
    for j in range(predictions.shape[2]):
        ft_maes.append(np.mean(np.abs(y_test[:,i,j]-predictions[:,i,j]), axis=0))
    ft_maes = np.array(ft_maes)
    maes.append(ft_maes)
maes = np.array(maes)

In [89]:
future_predictions = model.predict(future)

In [90]:
figs, axs = plt.subplots(len(FEATURES), 1, sharex=True, figsize=(17,17))
for i, col in enumerate(FEATURES):
    axs[i].plot(np.arange(len(future[0,:,i])), future[0,:,i])
    axs[i].plot(np.arange(len(future[0,:,i]), len(future[0,:,i])+TELESCOPE), future_predictions[0,:,i], color='orange')
    axs[i].fill_between(
        np.arange(len(future[0,:,i]), len(future[0,:,i])+TELESCOPE), 
        future_predictions[0,:,i]+maes[:,i], 
        future_predictions[0,:,i]-maes[:,i], 
        color='orange', alpha=.3)
    axs[i].set_title(col)
    #axs[i].set_ylim(0,1)
plt.show()

### Saving The Model

In [91]:
model.save('autoencoder')